<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=722a7c3497e3cbfbd7765edb38b5dfc5fd9dfec61b9fc3d2d431e69cf04f8efc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-18 14:43:36
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: -69.28 K (-0.48%)
Current PnL: -20.77 L (-13.61%)
CY Booked + Current PnL: -6.81 L (-4.46%)
-------------------
Total profit:  1.58 L
Total loss:  -22.35 L
-------------------
Total Booked + Current PnL: 20.17 L (16.06%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.62%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.37 C
Est FTT PnL: 91.59 L (63.13%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,149.44,84.0,M-SC,0.68,163482.0,-11077.0,11117.0,-0.22,-6.35,6.80,0.02,234.0,-1.00,1.13,42.20,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.64,49.0,M-SC,3.60,86460.0,-14317.0,14413.0,-0.65,-14.21,16.67,0.09,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-14.73,50.0,X-MC,3.67,89055.0,-1623.0,16529.0,0.42,-1.79,18.56,16.44,63.0,-0.10,0.61,18.02,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.07,57.0,X-MC,7.55,188799.0,14479.0,20730.0,-0.63,8.31,10.98,20.19,68.0,0.70,1.30,24.68,X40,ATH,INSURANCE
84,VOLTAS,1530.00,-1.91,54.0,X-MC,3.68,208440.0,16698.0,21052.0,1.14,8.71,10.10,19.69,78.0,0.79,1.44,16.10,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,112.74,69.0,H-SC,16.33,144331.0,-74165.0,102360.0,10.07,-33.94,70.92,12.90,161.0,-0.72,0.99,20.01,OX40N,NTT,FINANCE
64,SATIN,274.00,-22.47,63.0,H-SC,2.43,238729.0,-37555.0,180766.0,1.98,-13.59,75.72,51.83,146.0,-0.21,1.65,16.30,XY24,NTT,FINANCE
51,MASFIN,398.61,-16.44,59.0,H-SC,7.17,95100.0,-2880.0,24479.0,1.42,-2.94,25.74,22.05,164.0,-0.12,0.66,38.23,XR,ATH,FINANCE
12,BERGEPAINT,680.00,-2.91,73.0,X-MC,2.61,237350.0,10011.0,38047.0,1.38,4.40,16.03,21.14,74.0,0.26,1.64,33.78,XY24,NTT,PAINTS
66,SHALBY,327.00,1051.52,39.0,M-SC,17.04,156618.0,-24525.0,69993.0,1.25,-13.54,44.69,25.10,250.0,-0.35,1.08,25.51,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SURYODAY,240.0,55.70,45.0,H-SC,13.16,142055.0,-37016.0,102748.0,-3.62,-20.67,72.33,36.71,167.0,-0.36,0.98,40.95,XR,NTT,BANKS
81,VAIBHAVGBL,521.0,66.94,53.0,H-SC,10.18,144863.0,-37912.0,148977.0,-3.24,-20.74,102.84,60.77,159.0,-0.25,1.00,30.46,XR,NTT,JEWELLERY
13,BSOFT,831.7,-11.87,54.0,H-SC,8.32,96568.0,-44085.0,115515.0,-3.11,-31.34,119.62,50.79,151.0,-0.38,0.67,12.44,XR,ATH,IT
56,QUESS,986.0,-53.17,32.0,X-SC,37.20,46336.0,-18670.0,169599.0,-2.78,-28.72,366.02,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
8,AWL,485.0,-59.04,46.0,X-MC,3.11,255415.0,-46336.0,207755.0,-2.73,-15.36,81.34,53.50,58.0,-0.22,1.76,13.45,XY24,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,47.0,M-SC,10.09,147253.0,-391.0,159711.0,-0.72,-0.26,108.46,107.91,216.0,-0.00,1.02,2.73,OX40N,ATH,CABLES
70,STARHEALTH,761.00,36.05,66.0,H-SC,15.64,280160.0,-963.0,133832.0,-1.01,-0.34,47.77,47.26,171.0,-0.01,1.93,50.10,XY24,NTT,INSURANCE
86,WIPRO,420.00,-15.15,47.0,M-LC,5.58,149625.0,-1320.0,110782.0,-1.11,-0.87,74.04,72.51,99.0,-0.01,1.03,5.10,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,47.0,M-SC,10.09,147253.0,-391.0,159711.0,-0.72,-0.26,108.46,107.91,216.0,-0.00,1.02,2.73,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,149.44,84.0,M-SC,0.68,163482.0,-11077.0,11117.0,-0.22,-6.35,6.80,0.02,234.0,-1.00,1.13,42.20,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.64,49.0,M-SC,3.60,86460.0,-14317.0,14413.0,-0.65,-14.21,16.67,0.09,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.17,46.0,H-SC,2.22,221850.0,-47817.0,84148.0,-0.48,-17.73,37.93,13.47,158.0,-0.57,1.53,12.80,XY24,NTT,PAINTS
44,JCHAC,2282.00,19986.96,64.0,M-SC,1.45,98112.0,-29693.0,29679.0,0.08,-23.23,30.25,-0.01,233.0,-1.00,0.68,11.10,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,21.85,59.0,M-SC,36.45,164277.0,15057.0,129122.0,-0.62,10.09,78.60,96.62,245.0,0.12,1.13,77.78,XY25,NTT,FINANCE
38,INDIAMART,4810.62,-47.96,49.0,H-SC,4.57,129479.0,6143.0,125478.0,-1.31,4.98,96.91,106.72,139.0,0.05,0.89,29.52,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.96,49.0,H-SC,4.57,129479.0,6143.0,125478.0,-1.31,4.98,96.91,106.72,139.0,0.05,0.89,29.52,AR,ATH,MISC
25,FINCABLES,1641.55,-18.93,47.0,M-SC,10.09,147253.0,-391.0,159711.0,-0.72,-0.26,108.46,107.91,216.0,-0.00,1.02,2.73,OX40N,ATH,CABLES
86,WIPRO,420.00,-15.15,47.0,M-LC,5.58,149625.0,-1320.0,110782.0,-1.11,-0.87,74.04,72.51,99.0,-0.01,1.03,5.10,XR,NTT,IT
85,WHIRLPOOL,2270.00,-43.13,35.0,M-SC,8.45,111412.0,-3386.0,97430.0,-1.99,-2.95,87.45,81.92,236.0,-0.03,0.77,31.89,XR,NTT,DURABLES
87,ZYDUSLIFE,1286.17,-15.85,33.0,H-MC,5.41,205431.0,-4143.0,78803.0,-0.56,-1.98,38.36,35.63,119.0,-0.05,1.42,14.37,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.35,83813.0,-44857.0,90158.0,-0.47,-34.86,107.57,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
77,TMPV,1065.00,-20.63,25.0,X-LC,0.21,243594.0,-32816.0,453986.0,-0.21,-11.87,186.37,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,-20.15,29.0,X-MC,5.83,187043.0,-13304.0,43805.0,-1.51,-6.64,23.42,15.23,66.0,-0.30,1.29,23.47,X40N,NTT,REALTY
23,DMART,5391.45,-17.97,29.0,X-LC,5.23,200135.0,-10272.0,69447.0,-0.83,-4.88,34.70,28.12,19.0,-0.15,1.38,17.59,X40N,ATH,FMCG
34,HONAUT,58357.33,-26.34,29.0,X-SC,1.14,104775.0,-23163.0,70294.0,-1.48,-18.11,67.09,36.84,90.0,-0.33,0.72,7.26,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-20.63,25.0,X-LC,0.21,243594.0,-32816.0,453986.0,-0.21,-11.87,186.37,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,45.0,X-LC,0.90,198534.0,-1604.0,22494.0,-0.27,-0.80,11.33,10.44,5.0,-0.07,1.37,4.72,X40,NTT,FMCG
34,HONAUT,58357.33,-26.34,29.0,X-SC,1.14,104775.0,-23163.0,70294.0,-1.48,-18.11,67.09,36.84,90.0,-0.33,0.72,7.26,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.00,67.0,X-MC,1.19,184766.0,8028.0,46709.0,0.18,4.54,25.28,30.97,75.0,0.17,1.27,17.67,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-0.34,64.0,H-LC,1.67,162285.0,-23810.0,71292.0,0.41,-12.79,43.93,25.51,49.0,-0.33,1.12,19.59,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.35,83813.0,-44857.0,90158.0,-0.47,-34.86,107.57,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
56,QUESS,986.00,-53.17,32.0,X-SC,37.20,46336.0,-18670.0,169599.0,-2.78,-28.72,366.02,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
54,PAGEIND,51769.92,-29.20,36.0,X-MC,7.92,157480.0,-6260.0,49606.0,-0.25,-3.82,31.50,26.47,55.0,-0.13,1.09,0.00,X40,ATH,APPARELS
77,TMPV,1065.00,-20.63,25.0,X-LC,0.21,243594.0,-32816.0,453986.0,-0.21,-11.87,186.37,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-19.01,32.0,X-MC,6.73,174922.0,-25584.0,122743.0,-0.68,-12.76,70.17,48.45,64.0,-0.21,1.21,0.87,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,986.00,-53.17,32.0,X-SC,37.20,46336.0,-18670.0,169599.0,-2.78,-28.72,366.02,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-41.73,38.0,X-SC,3.03,79496.0,-65664.0,141590.0,-1.18,-45.24,178.11,52.31,91.0,-0.46,0.55,6.41,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.35,83813.0,-44857.0,90158.0,-0.47,-34.86,107.57,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-14.73,50.0,X-MC,3.67,89055.0,-1623.0,16529.0,0.42,-1.79,18.56,16.44,63.0,-0.10,0.61,18.02,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.34,29.0,X-SC,1.14,104775.0,-23163.0,70294.0,-1.48,-18.11,67.09,36.84,90.0,-0.33,0.72,7.26,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-26.86,57.0,X-LC,8.34,290836.0,-55120.0,121831.0,-0.26,-15.93,41.89,19.28,1.0,-0.45,2.00,7.52,X40,ATH,IT
41,INFY,2275.00,-21.03,50.0,X-LC,3.85,317327.0,4439.0,167263.0,-1.18,1.42,52.71,54.87,2.0,0.03,2.19,9.82,X40,BTT,IT
77,TMPV,1065.00,-20.63,25.0,X-LC,0.21,243594.0,-32816.0,453986.0,-0.21,-11.87,186.37,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
83,VBL,671.64,-18.42,47.0,X-LC,6.03,292300.0,-23542.0,136212.0,-0.77,-7.45,46.60,35.67,4.0,-0.17,2.01,5.37,X40N,ATH,FMCG
43,ITC,452.00,-40.74,45.0,X-LC,0.90,198534.0,-1604.0,22494.0,-0.27,-0.80,11.33,10.44,5.0,-0.07,1.37,4.72,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7661.11,65.0,L-SC,17.68,85177.0,-8633.0,85254.0,-0.67,-9.20,100.09,81.67,271.0,-0.10,0.59,64.95,XR,NTT,CERAMICS
51,MASFIN,398.61,-16.44,59.0,H-SC,7.17,95100.0,-2880.0,24479.0,1.42,-2.94,25.74,22.05,164.0,-0.12,0.66,38.23,XR,ATH,FINANCE
71,SURYODAY,240.00,55.70,45.0,H-SC,13.16,142055.0,-37016.0,102748.0,-3.62,-20.67,72.33,36.71,167.0,-0.36,0.98,40.95,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,66.94,53.0,H-SC,10.18,144863.0,-37912.0,148977.0,-3.24,-20.74,102.84,60.77,159.0,-0.25,1.00,30.46,XR,NTT,JEWELLERY
66,SHALBY,327.00,1051.52,39.0,M-SC,17.04,156618.0,-24525.0,69993.0,1.25,-13.54,44.69,25.10,250.0,-0.35,1.08,25.51,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,149.44,84.0,M-SC,0.68,163482.0,-11077.0,11117.0,-0.22,-6.35,6.80,0.02,234.0,-1.00,1.13,42.20,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.49,84.0,X-LC,10.15,255490.0,3722.0,49003.0,0.69,1.48,19.18,20.95,31.0,0.08,1.76,37.64,X40,ATH,PAINTS
81,VAIBHAVGBL,521.00,66.94,53.0,H-SC,10.18,144863.0,-37912.0,148977.0,-3.24,-20.74,102.84,60.77,159.0,-0.25,1.00,30.46,XR,NTT,JEWELLERY
13,BSOFT,831.70,-11.87,54.0,H-SC,8.32,96568.0,-44085.0,115515.0,-3.11,-31.34,119.62,50.79,151.0,-0.38,0.67,12.44,XR,ATH,IT
40,INDUSINDBK,1800.00,-696.43,68.0,L-MC,5.94,48316.0,-28890.0,54283.0,-0.75,-37.42,112.35,32.89,259.0,-0.53,0.33,33.06,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.51
1,25,44.30
2,50,75.10


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.78
MC    28.63
LC    24.58
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.00
X40      20.47
X40N     11.52
XY25     10.13
XR        9.50
AR        9.04
OX40N     7.73
X200      1.79
SR        1.00
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-MC    22.26
X-LC    19.66
M-SC    13.30
X-SC     7.71
H-MC     4.68
H-LC     2.91
L-SC     1.42
M-MC     1.36
M-LC     1.03
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.54,-4.79,38.68
IT,13.30,-15.46,75.11
FINANCE,10.61,-10.38,62.29
MISC,7.42,-20.54,81.34
PAINTS,6.06,-5.14,20.76
ELECTRICAL,5.66,-10.14,49.18
INSURANCE,4.50,3.30,30.79
PHARMA,3.40,-1.20,35.15
AUTO,3.36,-17.81,130.76


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3380629.0,21
XR,1279386.0,13
AR,1259821.0,10
X40,998085.0,14
X40N,700465.0,9
OX40N,672423.0,10
XY25,463800.0,7
SR,274321.0,2
MH,66110.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3469407.0,25
M-SC,1424858.0,16
X-LC,1214866.0,11
X-MC,1164920.0,16
X-SC,845027.0,8
H-MC,399893.0,3
L-SC,255656.0,3
H-LC,135670.0,2
M-LC,110782.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1166744.0      6
           AR         848369.0      5
           XR         799946.0      7
M-SC       XY24       754317.0      6
X-LC       XY24       522430.0      2
           X40        411574.0      5
X-MC       X40        411481.0      7
X-SC       XY24       403804.0      3
X-MC       XY24       349407.0      3
H-SC       OX40N      313917.0      4
           SR         274321.0      2
M-SC       OX40N      271606.0      5
X-SC       X40N       266193.0      3
X-MC       X40N       228613.0      4
H-MC       AR         215966.0      2
X-LC       X40N       205659.0      2
H-MC       XY24       183927.0      1
X-MC       XY25       175419.0      2
X-SC       X40        175030.0      2
L-SC       XR         168756.0      2
M-SC       XR         145619.0      2
           XY25       129122.0      1
           AR         124194.0      2
M-LC       XR         110782.0      1
L-SC       OX40N       86900.0      1
X-LC       XY25        75203.0      2
H-LC       AR          71292.0      1
H-SC       MH          66110.0      1
H-LC       X200        64378.0      1
L-MC       XR          54283.0      1
M-MC       XY25        47494.0      1
L-LC       XY25        36562.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
